In [1]:
import torch

In [2]:
# 定义数据
x = torch.tensor(10.0)
y = torch.tensor(3.0)
print(x)
print(y)

tensor(10.)
tensor(3.)


In [3]:
# 初始化参数
w = torch.rand(1, 1, requires_grad=True)
b = torch.rand(1, 1, requires_grad=True)

In [4]:
# 前向传播得到输出值
z = w * x + b
print(z)

tensor([[4.3294]], grad_fn=<AddBackward0>)


In [5]:
# 叶子节点会一直保存梯度，非叶子节点不会存储
print(x.is_leaf)
print(w.is_leaf)
print(b.is_leaf)
print(y.is_leaf)
print(z.is_leaf)

True
True
True
True
False


In [7]:
# 设置损失函数
loss = torch.nn.MSELoss()
loss_value = loss(z, y)
print(loss_value)
print(loss_value.is_leaf)

tensor(1.7674, grad_fn=<MseLossBackward0>)
False


In [8]:
# 反向传播
loss_value.backward()

In [9]:
# 查看梯度
print(w.grad)
print(b.grad)

tensor([[26.5888]])
tensor([[2.6589]])


自动微分的关键就是记录节点的数据与运算。数据记录在张量的`data`属性中，计算记录在张量的`grad_fn`属性中。


计算图根据搭建方式可分为静态图和动态图，`PyTorch`是动态图机制，在计算的过程中逐步搭建计算图，同时对每个`Tensor`都存储`grad_fn`供自动微分使用。


若设置张量参数`requires_grad=True`，则`PyTorch`会追踪素有基于该张量的操作，并在反向传播时计算其梯度。以来于叶子节点的节点，`requires_grad`默认为`True`。当计算到根节点后，在根节点调用`backward()`方法即可反向传播计算计算图中所有节点的梯度。


非叶子节点的梯度在反向传播之后会被释放掉（除非设置参数`retain_grad=True`）。而叶子节点的梯度在反向传播之后会保留（累积）。通常需要`optimizer.zero_grad()`清零参数的梯度。


有时我们希望将某些计算移动到计算图之外，可以使用`Tensor.detach()`返回一个新的变量，该标量与原变量具有相同的值，但丢失计算图中如何计算原变量的信息。换句话说，梯度不会在该变量处继续乡下传播。